In [1]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from pandas import *
import numpy as np
import jieba as jb
import re

In [2]:
labels = [];datas = [];
fr = open('toutiao_cat_data.txt',encoding='utf-8')
for line in fr.readlines():
    lineArr = line.strip().split("_!_")#strip()删除换行，用split("_!_")把每行文本按照_!_分开
    labels.append(lineArr[2])#标签
    datas.append(lineArr[3] + lineArr[4])#特征：标题+新闻关键词
#用DataFrame()存储
df = pandas.DataFrame()
df['data'] = datas
df['label'] = labels

In [3]:
df

,data,label
0,"京城最值得你来场文化之旅的博物馆保利集团,马未都,中国科学技术馆,博物馆,新中国",news_culture
1,发酵床的垫料种类有哪些？哪种更好？,news_culture
2,上联：黄山黄河黄皮肤黄土高原。怎么对下联？,news_culture
3,林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？,news_culture
4,黄杨木是什么树？,news_culture
...,...,...
382683,A10处理器iPhone SE二代值得期待吗？,news_tech
382684,先进战机叛逃将带来重大损失，美军如何防止F22飞行员驾机叛逃？,news_military
382685,"又一国领导人放话，只要普京下令，数万大军“碾压”美国白宫！以色列,普京,俄罗斯,叙利亚,车臣",news_world
382686,如何看待美国总统连续撕毁美国签署的国际协议的举动？,news_world


In [4]:
#列表中添加类型id
df['id'] = df['label'].factorize()[0]
id_df = df[['label', 'id']].drop_duplicates().sort_values('id').reset_index(drop=True)

In [5]:
df

,data,label,id
0,"京城最值得你来场文化之旅的博物馆保利集团,马未都,中国科学技术馆,博物馆,新中国",news_culture,0
1,发酵床的垫料种类有哪些？哪种更好？,news_culture,0
2,上联：黄山黄河黄皮肤黄土高原。怎么对下联？,news_culture,0
3,林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？,news_culture,0
4,黄杨木是什么树？,news_culture,0
...,...,...,...
382683,A10处理器iPhone SE二代值得期待吗？,news_tech,7
382684,先进战机叛逃将带来重大损失，美军如何防止F22飞行员驾机叛逃？,news_military,8
382685,"又一国领导人放话，只要普京下令，数万大军“碾压”美国白宫！以色列,普京,俄罗斯,叙利亚,车臣",news_world,10
382686,如何看待美国总统连续撕毁美国签署的国际协议的举动？,news_world,10


### 中文文本预处理

In [6]:
#数据清洗
#删除标点
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line
#删除停用词
def stopwordslist():  
    stopwords = [line.strip() for line in open("chineseStopWords.txt", 'r', encoding='utf-8').readlines()]  
    return stopwords  
stopwords = stopwordslist()

In [7]:
df['clean_data'] = df['data'].apply(remove_punctuation)#删除标点
df['cut_data'] = df['clean_data'].apply(lambda x: " ".join([w for w in list(jb.cut(x)) if w not in stopwords]))#分词
df

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LY\AppData\Local\Temp\jieba.cache
Loading model cost 0.814 seconds.
Prefix dict has been built successfully.


,data,label,id,clean_data,cut_data
0,"京城最值得你来场文化之旅的博物馆保利集团,马未都,中国科学技术馆,博物馆,新中国",news_culture,0,京城最值得你来场文化之旅的博物馆保利集团马未都中国科学技术馆博物馆新中国,京城 值得 来场 文化 之旅 博物馆 保利 集团 马未 中国科学技术馆 博物馆 新 中国
1,发酵床的垫料种类有哪些？哪种更好？,news_culture,0,发酵床的垫料种类有哪些哪种更好,发酵 床 垫料 种类 种 更好
2,上联：黄山黄河黄皮肤黄土高原。怎么对下联？,news_culture,0,上联黄山黄河黄皮肤黄土高原怎么对下联,上联 黄山 黄河 黄皮肤 黄土高原 下联
3,林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？,news_culture,0,林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣,林徽因 理由 拒绝 徐志摩 选择 梁思成 终身伴侣
4,黄杨木是什么树？,news_culture,0,黄杨木是什么树,黄杨木 树
...,...,...,...,...,...
382683,A10处理器iPhone SE二代值得期待吗？,news_tech,7,A10处理器iPhoneSE二代值得期待吗,A10 处理器 iPhoneSE 二代 值得 期待
382684,先进战机叛逃将带来重大损失，美军如何防止F22飞行员驾机叛逃？,news_military,8,先进战机叛逃将带来重大损失美军如何防止F22飞行员驾机叛逃,先进 战机 叛逃 带来 重大损失 美军 防止 F22 飞行员 驾机 叛逃
382685,"又一国领导人放话，只要普京下令，数万大军“碾压”美国白宫！以色列,普京,俄罗斯,叙利亚,车臣",news_world,10,又一国领导人放话只要普京下令数万大军碾压美国白宫以色列普京俄罗斯叙利亚车臣,一国 领导人 放 话 普京 下令 数万 大军 碾压 美国白宫 以色列 普京 俄罗斯 叙利亚 车臣
382686,如何看待美国总统连续撕毁美国签署的国际协议的举动？,news_world,10,如何看待美国总统连续撕毁美国签署的国际协议的举动,看待 美国 总统 连续 撕毁 美国 签署 国际 协议 举动


In [8]:
#划分训练集、测试集
X_train, X_test, y_train, y_test = train_test_split(df['cut_data'], df['id'], random_state = 0)

In [9]:
#生成词频向量和TF-IDF向量
#训练集集
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
#测试集
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

### 手写朴素贝叶斯分类器

In [10]:
#朴素贝叶斯函数
#train_tfidf文档词向量矩阵
#trainlabel每条文档类别标签
def trainByes(train_tfidf,trainlabel):
    numTrainDocs = train_tfidf.shape[0] #训练的文档条数 6451
    numWords = train_tfidf.shape[1] #计算每条文档的词个数,这里是tfidf向量的维度 23896
    
    #定义总共15个类别
    Types = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
    types = len(Types) #15
    
    #文档属于各个类别的概率
    pFileType = []
    for i in range(types):
        pFileType.append(trainlabel.value_counts()[i] / float(numTrainDocs))#trainlabel.value_counts()[i]对类别为i的文档进行计数
    
    #应用拉普拉斯平滑，初始化所有词出现数为1，避免某一个概率值为0   
    pWord = np.ones([types, train_tfidf.shape[1]]) # 维度：(15,23896),拉普拉斯平滑

    #分母初始化为2
    pDemo = []
    for i in range(types):
        pDemo.append(2.0)
        
    # 各个类的矩阵相加
    #y_train的index是类似3697/136/24775...的形式，不是train_tfidf的维度顺序，因此每遍历一个y_train索引就对应train_tfidf矩阵行数+1
    count = -1
    for i in y_train.index:
        count += 1
        for j in Types:
            if y_train[i] == Types[j]:
                pWord[j] += train_tfidf[count]
                pDemo[j] += np.sum(train_tfidf[count])

    #计算各类条件概率组
    #结果取对数，避免大量小数数相乘造成的影响
    pVect = []#15个元素对应每一种类别的条件概率
    for i in range(types):
        pVect.append(np.log(pWord[i]/pDemo[i]))

    return pVect,pFileType

In [11]:
pVect,pFileType = trainByes(X_train_tfidf,y_train)

In [12]:
#用朴素贝叶斯函数进行分类
def classifyBayes(test_tfidf,pVect,pFileType):
    #将pVect转为矩阵方便计算
    pVectMat = np.mat(pVect)
    
    import numpy.matlib
    Types = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
    types = len(Types) #15     
    
    #采用矩阵计算
    #(数据条数,词条向量维度)*(15,1词条向量维度.T) = (数据条数,15)
    temp = test_tfidf * pVectMat.T#进行转置，使得结果矩阵每行对应一个词条各个类别的概率
    for i in Types:
        temp[:,i] += np.log(pFileType[i]) 
    
    #即将矩阵每行概率最大的元素所对应的列索引作为对应的类别
    pClass = np.argmax(temp, axis=1)
            
    return pClass

In [13]:
y_pred = classifyBayes(X_test_tfidf,pVect,pFileType)

In [15]:
#评估结果
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print(accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=id_df['label'].values))

0.36745338238983194
                    precision    recall  f1-score   support

      news_culture       0.97      0.51      0.67      7044
news_entertainment       0.95      0.19      0.32      9901
       news_sports       0.98      0.38      0.54      9286
      news_finance       0.84      0.12      0.21      6869
        news_house       0.93      0.55      0.70      4410
          news_car       0.96      0.40      0.56      8975
          news_edu       0.93      0.39      0.55      6797
         news_tech       0.91      0.50      0.65     10464
     news_military       0.89      0.36      0.51      6185
       news_travel       0.90      0.27      0.41      5272
        news_world       0.87      0.43      0.57      6621
  news_agriculture       0.90      0.24      0.38      4791
         news_game       0.95      0.45      0.61      7460
             stock       0.00      0.86      0.00        97
        news_story       0.81      0.27      0.40      1500

          accuracy